# Gerenciando pacotes

In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
#!apt upgrade

Defaulting to user installation because normal site-packages is not writeable
Lendo listas de pacotes... Pronto
E: Não foi possível abrir arquivo de trava /var/lib/apt/lists/lock - open (13: Permissão negada)
E: Impossível criar acesso exclusivo ao directório /var/lib/apt/lists/
W: Problema ao remover o link do ficheiro /var/cache/apt/pkgcache.bin - RemoveCaches (13: Permissão negada)
W: Problema ao remover o link do ficheiro /var/cache/apt/srcpkgcache.bin - RemoveCaches (13: Permissão negada)
E: Não foi possível abrir arquivo de trava /var/lib/dpkg/lock-frontend - open (13: Permissão negada)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [6]:
import sys
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

# Extraindo links dos Hoteis

Aqui, vou raspar os dados da página que contém os 25 melhores hoteis do Brasil

## Filtrando um caso

In [7]:
driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')

#link da lista 
url = 'https://www.tripadvisor.com.br/TravelersChoice-Hotels-cTop-g294280'
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

/tmp/ipykernel_9021/717171602.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')


In [8]:
all_hotel = soup.find_all("div", attrs={"class", "winnerName"})

In [9]:
len(all_hotel)

25

In [10]:
all_hotel[0:3]

[<div class="winnerName">
 <div class="mainName extra">
 <a href="/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink', 'Top-294280', 1, this.href);" target="_blank">1. Hotel Colline de France</a>
 </div>
 <div class="smaller">
 <a href="/Tourism-g303536-Gramado_State_of_Rio_Grande_do_Sul-Vacations.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink2', 'Top-294280', 1, this.href);" target="_blank">Gramado, Brasil</a>
 </div>
 </div>,
 <div class="winnerName">
 <div class="mainName extra">
 <a href="/Hotel_Review-g303536-d7255333-Reviews-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink', 'Top-294280', 2, this.href);" target="_blank">2. Hotel Valle D’Incanto</a>
 </div>
 <div class="smaller">
 <a href="/Tourism-g303536-Gramado_State_of_Rio_Grande_do_Sul-Vacations.html" onclick="ta.setEvtCookie('PhotoVi

In [11]:
all_hotel[0].find_all("a")

[<a href="/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink', 'Top-294280', 1, this.href);" target="_blank">1. Hotel Colline de France</a>,
 <a href="/Tourism-g303536-Gramado_State_of_Rio_Grande_do_Sul-Vacations.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink2', 'Top-294280', 1, this.href);" target="_blank">Gramado, Brasil</a>]

In [12]:
all_hotel[0].find_all("a")[0]['href'] #OK! Me dá o link

'/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html'

In [13]:
all_hotel[0].find_all("a")[0]

<a href="/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink', 'Top-294280', 1, this.href);" target="_blank">1. Hotel Colline de France</a>

In [14]:
all_hotel[0].find_all("a")[1]

<a href="/Tourism-g303536-Gramado_State_of_Rio_Grande_do_Sul-Vacations.html" onclick="ta.setEvtCookie('PhotoViewer', 'WinnerNameLink2', 'Top-294280', 1, this.href);" target="_blank">Gramado, Brasil</a>

In [15]:
re.search(r'target="_blank">.?.?.?(.*)<\/a>', str(all_hotel[0].find_all("a")[0])).group(1)

'Hotel Colline de France'

In [16]:
re.search(f'target="_blank">(.*), Brasil<\/a>', str(all_hotel[0].find_all("a")[1])).group(1)

'Gramado'

## Generalizando e gerando dataframe

In [17]:
data = []
for pos, prov in enumerate(all_hotel):
    endereco = prov.find_all("a")[0]['href']
    cidade = re.search(f'target="_blank">(.*), Brasil<\/a>', str(prov.find_all("a")[1])).group(1)
    nome = re.search(r'target="_blank">[0-9\. ]*(.*)<\/a>', str(prov.find_all("a")[0])).group(1)
    data.append([nome, cidade, "https://www.tripadvisor.com.br"+endereco])

In [18]:
df_lista = pd.DataFrame(data, columns = ['nome_hotel', 'cidade', 'link'])

In [19]:
df_lista

,nome_hotel,cidade,link
0,Hotel Colline de France,Gramado,https://www.tripadvisor.com.br/Hotel_Review-g3...
1,Hotel Valle D’Incanto,Gramado,https://www.tripadvisor.com.br/Hotel_Review-g3...
2,Hotel Ritta Höppner,Gramado,https://www.tripadvisor.com.br/Hotel_Review-g3...
3,Hotel Refugio da Montanha,Gramado,https://www.tripadvisor.com.br/Hotel_Review-g3...
4,Hotel Estalagem St Hubertus,Gramado,https://www.tripadvisor.com.br/Hotel_Review-g3...
5,Hotel Le Renard,Campos do Jordão,https://www.tripadvisor.com.br/Hotel_Review-g3...
6,Carballo Hotel &amp; Spa,Campos do Jordão,https://www.tripadvisor.com.br/Hotel_Review-g3...
7,Porto Pacuíba Hotel,Ilhabela,https://www.tripadvisor.com.br/Hotel_Review-g6...
8,Salinas Maragogi All Inclusive Resort,Maragogi,https://www.tripadvisor.com.br/Hotel_Review-g6...
9,Casas Brancas Boutique Hotel &amp; Spa,Armação dos Búzios,https://www.tripadvisor.com.br/Hotel_Review-g3...


In [20]:
links_hoteis = df_lista['link'].to_list()

In [21]:
links_hoteis

['https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d778984-Reviews-Hotel_Ritta_Hoppner-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d12398420-Reviews-Hotel_Refugio_da_Montanha-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d1837047-Reviews-Hotel_Estalagem_St_Hubertus-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303607-d1958657-Reviews-Hotel_Le_Renard-Campos_Do_Jordao_State_of_Sao_Paulo.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303607-d6888786-Reviews-Carballo_Hotel_Spa-Campos_Do_Jordao_State_of_Sao_Paulo.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g6091

# Extraindo para um único hotel

In [22]:
links_hoteis[0]

'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html'

In [23]:
driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')

#link da lista 
driver.get(links_hoteis[0])

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

/tmp/ipykernel_9021/985259620.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')


In [27]:
soup

<html lang="pt-BR" xmlns:og="http://opengraphprotocol.org/schema/"><head><meta content="text/html; charset=utf-8" http-equiv="content-type"/><link href="https://static.tacdn.com/favicon.ico?v2" id="favicon" rel="icon" type="image/x-icon"/><link color="#000000" href="https://static.tacdn.com/img2/brand_refresh/application_icons/mask-icon.svg" rel="mask-icon" sizes="any"/><meta content="#34e0a1" name="theme-color"/><meta content="telephone=no" name="format-detection"/><script type="text/javascript">window.taRollupsAreAsync = true;</script><link crossorigin="" href="https://static.tacdn.com/css2/webfonts/TripSans/TripSans.css?v1.002" rel="stylesheet"/><link as="fetch" crossorigin="anonymous" href="/static/decodeKey.txt" rel="preload"/><title>HOTEL COLLINE DE FRANCE (GRAMADO): 3.177 fotos, comparação de preços e 2.556 avaliações - Tripadvisor</title><meta content="TripAdvisor" property="al:ios:app_name"/><meta content="284876795" property="al:ios:app_store_id"/><meta content="284876795" na

## Conseguindo link por página

In [28]:
prox_page = soup.find_all("a", attrs={"class", "ui_button nav next primary"})

In [29]:
prox_page

[<a class="ui_button nav next primary" href="/Hotel_Review-g303536-d13871137-Reviews-or5-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html">Próximas</a>]

In [30]:
prox_page[0]['href']

'/Hotel_Review-g303536-d13871137-Reviews-or5-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html'

In [31]:
link_split = prox_page[0]['href'].split('-or5-')

In [32]:
for num_coment_inf in range(0, 21, 5):
    if num_coment_inf == 0:
        link = "https://www.tripadvisor.com.br" + link_split[0] + '-' + link_split[1]
    else:
        link = "https://www.tripadvisor.com.br" + link_split[0] + '-' + 'or' + str(num_coment_inf) + '-' + link_split[1]
    print(link)

https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html
https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or5-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html
https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or10-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html
https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or15-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html
https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or20-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html


## Dados para uma página


In [33]:
link

'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or20-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html'

In [34]:
driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')

#link da lista 
driver.get(link)

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

/tmp/ipykernel_9021/1875402916.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')


### Notas

In [35]:
nota_soup = soup.find_all("div", attrs={"class", "Hlmiy F1"})

In [36]:
nota_soup

[<div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_50"></span></div>,
 <div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_50"></span></div>,
 <div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_50"></span></div>,
 <div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_50"></span></div>,
 <div class="Hlmiy F1" data-test-target="review-rating"><span class="ui_bubble_rating bubble_50"></span></div>]

In [37]:
notas = []
for prov in nota_soup:
    nota = int(re.findall(r'rating bubble_(..)', str(prov))[0]) // 10
    notas.append(nota)
notas

[5, 5, 5, 5, 5]

### Texto da avaliação

In [38]:
coment_soup = soup.find_all("q", attrs={"class", "QewHA H4 _a"})

In [39]:
coment_soup

[<q class="QewHA H4 _a"><span>Hotel é maravilhoso em varios aspectos. Além de lindo, somos muito bem cuidados desde a porta de entrada. Devo ressaltar aqui o quanto fiquei impressionada com a atenção que minha bebe (na época com 7 meses) recebeu. Foi muito paparicada por todos (infelizmente nao vou saber o nome de todos os que nos trataram tao bem mas devo citar aqui o Luca, que foi um querido com minha pequena). Ja na chegada deram uma pelúcia para ela. O bercinho que montaram no quarto era lindo e tinha uma banheirinha também.</span></q>,
 <q class="QewHA H4 _a"><span>Desde a chegada , recepção , atendimento de todos uma educação , pensa em um a café da manhã igual da Rainha da Inglaterra,  no jantar pratos maravilhosos, mas o atendimento de todos , começando pela recepção  a zeladoria , já fiquei em Hoteis 5 estrelas em muitos países e realmente igual ao CollinedeFrance nenhum Superou ele .</span></q>,
 <q class="QewHA H4 _a"><span>Tudo perfeito, cada detalhe, tudo muito perfeito! F

In [40]:
coments = []
for prov in coment_soup:
    coment = re.findall(r'<span>(.*)<\/span>', str(prov))
    coments.append(coment[0])
coments

['Hotel é maravilhoso em varios aspectos. Além de lindo, somos muito bem cuidados desde a porta de entrada. Devo ressaltar aqui o quanto fiquei impressionada com a atenção que minha bebe (na época com 7 meses) recebeu. Foi muito paparicada por todos (infelizmente nao vou saber o nome de todos os que nos trataram tao bem mas devo citar aqui o Luca, que foi um querido com minha pequena). Ja na chegada deram uma pelúcia para ela. O bercinho que montaram no quarto era lindo e tinha uma banheirinha também.',
 'Desde a chegada , recepção , atendimento de todos uma educação , pensa em um a café da manhã igual da Rainha da Inglaterra,  no jantar pratos maravilhosos, mas o atendimento de todos , começando pela recepção  a zeladoria , já fiquei em Hoteis 5 estrelas em muitos países e realmente igual ao CollinedeFrance nenhum Superou ele .',
 'Tudo perfeito, cada detalhe, tudo muito perfeito! Ficamos apenas uma diária mas pareceu uma semana, aproveitamos muito! Espero voltar logo.  Obrigada equip

### Titulo da avaliação

In [34]:
titulo_soup = soup.find_all("div", attrs={"class", "KgQgP MC _S b S6 H5 _a"})
titulo_soup

[<div class="KgQgP MC _S b S6 H5 _a" data-test-target="review-title" dir="ltr"><a class="Qwuub" href="/ShowUserReviews-g303536-d13871137-r844261643-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html"><span><span>Atendimento perfeito, conforto e ambiente são maravilhosos, se percebe a excelência em cada detalhe!</span></span></a></div>,
 <div class="KgQgP MC _S b S6 H5 _a" data-test-target="review-title" dir="ltr"><a class="Qwuub" href="/ShowUserReviews-g303536-d13871137-r844261218-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html"><span><span>CollinedeFrance</span></span></a></div>,
 <div class="KgQgP MC _S b S6 H5 _a" data-test-target="review-title" dir="ltr"><a class="Qwuub" href="/ShowUserReviews-g303536-d13871137-r844253576-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html"><span><span>Impressionante</span></span></a></div>,
 <div class="KgQgP MC _S b S6 H5 _a" data-test-target="review-title" dir="ltr"><a class="Qwuub" href="/ShowUserReviews

In [35]:
titulo = re.findall(r'<span><span>(.*)<\/span><\/span>', str(titulo_soup[0]))
titulo

['Atendimento perfeito, conforto e ambiente são maravilhosos, se percebe a excelência em cada detalhe!']

In [36]:
titulos = []
for prov in titulo_soup:
    titulo = re.findall(r'<span><span>(.*)<\/span><\/span>', str(prov))
    titulos.append(titulo[0])
titulos

['Atendimento perfeito, conforto e ambiente são maravilhosos, se percebe a excelência em cada detalhe!',
 'CollinedeFrance',
 'Impressionante',
 'INCRÍVEL',
 'Uma experiência!']

### Data avaliação

In [37]:
data_aval_soup = soup.find_all("div", attrs={"class", "cRVSd"})
data_aval_soup

[<div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/giulianof343">Giuliano F</a> escreveu uma avaliação jun. de 2022</span></div>,
 <div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/R7802CMdiegoa">Diego A</a> escreveu uma avaliação jun. de 2022</span></div>,
 <div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/Thuayres">Thuanny</a> escreveu uma avaliação jun. de 2022</span></div>,
 <div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/ju_abadio">Juliana Abadio</a> escreveu uma avaliação jun. de 2022</span></div>,
 <div class="cRVSd"><span><a class="ui_header_link uyyBf" href="/Profile/Nianicorrea">Nianicorrea</a> escreveu uma avaliação jun. de 2022</span></div>]

In [38]:
data_aval = re.findall(r'escreveu uma avaliação (.*)<\/span><\/div>', str(data_aval_soup[0]))
data_aval

['jun. de 2022']

In [39]:
data_avals = []
for prov in data_aval_soup:
    data_aval = re.findall(r'escreveu uma avaliação (.*)<\/span><\/div>', str(prov))
    data_avals.append(data_aval[0])
data_avals

['jun. de 2022',
 'jun. de 2022',
 'jun. de 2022',
 'jun. de 2022',
 'jun. de 2022']

### Data da estadia

In [40]:
data_esta_soup = soup.find_all("span", attrs={"class", "teHYY _R Me S4 H3"})
data_esta_soup

[<span class="teHYY _R Me S4 H3"><span class="usajM">Data da estadia:</span> junho de 2022</span>,
 <span class="teHYY _R Me S4 H3"><span class="usajM">Data da estadia:</span> junho de 2022</span>,
 <span class="teHYY _R Me S4 H3"><span class="usajM">Data da estadia:</span> junho de 2022</span>,
 <span class="teHYY _R Me S4 H3"><span class="usajM">Data da estadia:</span> junho de 2022</span>,
 <span class="teHYY _R Me S4 H3"><span class="usajM">Data da estadia:</span> junho de 2022</span>]

In [41]:
data_esta = re.findall(r'<\/span> (.*)<\/span>', str(data_esta_soup[0]))
data_esta

['junho de 2022']

In [42]:
data_estas = []
for prov in data_esta_soup:
    data_esta = re.findall(r'<\/span> (.*)<\/span>', str(prov))
    data_estas.append(data_esta[0])
data_estas

['junho de 2022',
 'junho de 2022',
 'junho de 2022',
 'junho de 2022',
 'junho de 2022']

### Definindo uma função geradora

In [43]:
def raspa_pagina(soup, tag_html, atributo, match_atributo, regex):
    
    
    data_soup = soup.find_all(tag_html, attrs={atributo, match_atributo})
    x  = []
    
    for prov in (data_soup):
        _ = re.findall(regex, str(prov))
        
        if _ != []:
            if regex == 'rating bubble_(..)':
                x.append(int(_[0]) // 10)
            else:
                x.append(_[0])
        else: 
            x.append("ERRO!!!")

    return x


In [44]:
raspa_pagina(soup, 'div', 'class', 'Hlmiy F1', 'rating bubble_(..)') # NOTAS

[5, 5, 5, 5, 5]

In [45]:
raspa_pagina(soup, 'q', 'class', 'QewHA H4 _a', '<span>(.*)<\/span>') #TEXTO

['O atendimento é sensacional, o ambiente é extremamente agradável, comida excelente, serviço de quarto excepcional, a massagem relaxante é maravilhosa e ainda estão instalando sauna e piscina. Ficar hospedado neste hotel é uma experiência incrível!!!',
 'A estrutura do hotel é espetacular, mas nada se compara ao atendimento, nunca vi nada igual, a comida é muito boa, o café da manhã é sensacional, mas realmente o ponto alto do hotel é o atendimento, equipe acima da média',
 'Toda a equipe é muito receptiva e gentil! Ganhamos muitos “mimos” no quarto (champanhe, doces…) café da manhã delicioso! Quarto muuuuito cheiroso, lençóis de ótima qualidade.  Tudo impecável, amei! Mais que recomendado à todos',
 'Simplesmente incrível! O lugar é lindo, mas o que faz toda diferença é o atendimento que é SURREAL. As pessoas são super gentis e simpáticas e prontas pra te ajudar no que precisar. Vc sente a preocupação do hotel com a experiência da hora que chega até a hora que vai embora.',
 'Excelên

In [46]:
raspa_pagina(soup, 'div', 'class', 'KgQgP MC _S b S6 H5 _a', '<span><span>(.*)<\/span><\/span>') #TITULO

['Atendimento perfeito, conforto e ambiente são maravilhosos, se percebe a excelência em cada detalhe!',
 'CollinedeFrance',
 'Impressionante',
 'INCRÍVEL',
 'Uma experiência!']

In [47]:
raspa_pagina(soup, 'div', 'class', 'cRVSd', 'escreveu uma avaliação (.*)<\/span><\/div>') #DATA


['jun. de 2022',
 'jun. de 2022',
 'jun. de 2022',
 'jun. de 2022',
 'jun. de 2022']

## Validando

In [48]:
links_hoteis #OK

['https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d778984-Reviews-Hotel_Ritta_Hoppner-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d12398420-Reviews-Hotel_Refugio_da_Montanha-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d1837047-Reviews-Hotel_Estalagem_St_Hubertus-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303607-d1958657-Reviews-Hotel_Le_Renard-Campos_Do_Jordao_State_of_Sao_Paulo.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303607-d6888786-Reviews-Carballo_Hotel_Spa-Campos_Do_Jordao_State_of_Sao_Paulo.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g6091

In [49]:
driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')

#link da lista 
driver.get(links_hoteis[0])

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

/tmp/ipykernel_73675/985259620.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')


In [50]:
raspa_pagina(soup, 'div', 'class', 'Hlmiy F1', 'rating bubble_(..)') # NOTAS

[5, 5, 5, 5, 5]

In [51]:
raspa_pagina(soup, 'q', 'class', 'QewHA H4 _a', '<span>(.*)<\/span>') #TEXTO

['Me senti num castelo com personagens contemporâneos! Logo na chegada você percebe que está num lugar diferenciado. Equipe preparada para atender de forma única, recebendo com delicadeza, atenção e entusiasmo, valorizando o hóspede e tudo o que ali está sendo feito para proporcionar momentos inesquecíveis! Final de tarde os hóspedes são recebidos ao som de um piano que fica ao lado da receptação, ambiente lindo, sofisticado e como se não bastasse, servem champanhe e doces delicados, ao lado de uma lareira, num espaço aconchegante e sofisticado. Tudo parece ter sido pensado com detalhes que transmitem aos hóspedes um efeito “nós fizemos a diferença por você ”. Chá, água saborizada  e champanhe a vontade! Na saída uma mensagem, água e madelaines para lhe acompanhar na viagem!</span><span class="_"> Incrível!!!',
 'Serviço extraordinário! Preocupação total com o hóspede nos pequenos detalhes.  Café da manhã estilo francês sensacional ! Atendimento personalizado para que o hóspede tenha u

In [52]:
raspa_pagina(soup, 'div', 'class', 'KgQgP MC _S b S6 H5 _a', '<span><span>(.*)<\/span><\/span>') #TITULO

['Coline de France experiência única!',
 'Experiência Diferenciada no Atendimento',
 'Sonho',
 'O HOTEL É PERFEITOOO, O ATENDIMENTO É SURREAL.',
 'Sensacional ']

In [53]:
raspa_pagina(soup, 'div', 'class', 'cRVSd', 'escreveu uma avaliação (.*)<\/span><\/div>') #DATA


['jun. de 2022',
 'jun. de 2022',
 'jun. de 2022',
 'jun. de 2022',
 'jun. de 2022']

In [54]:
driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')

#link da lista 
driver.get(links_hoteis[1])

soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

/tmp/ipykernel_73675/1958673681.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')


In [55]:
_nota = raspa_pagina(soup, 'div', 'class', 'Hlmiy F1', 'rating bubble_(..)') # NOTAS

In [56]:
_texto = raspa_pagina(soup, 'q', 'class', 'QewHA H4 _a', '<span>(.*)<\/span>') #TEXTO

In [57]:
_titulo = raspa_pagina(soup, 'div', 'class', 'KgQgP MC _S b S6 H5 _a', '<span><span>(.*)<\/span><\/span>') #TITULO

In [58]:
_data = raspa_pagina(soup, 'div', 'class', 'cRVSd', 'escreveu uma avaliação (.*)<\/span><\/div>') #DATA


In [59]:
teste = pd.DataFrame(np.transpose(np.array([_nota, _titulo, _texto, _data])))
teste

,0,1,2,3
0,5,Irretocável!,Esse hotel é simplesmente perfeito: desde as a...,Hoje
1,5,Sempre uma excelente experiência!,Retornamos ao hotel devido a excelente experiê...,fev. de 2022
2,5,Você precisa viver essa experiência nesse hote...,Vocês precisam se hospedar aqui!!!!! O hotel é...,jun. de 2022
3,5,Viagem em casal,Indescritível a experiência. Carinho e aconch...,jun. de 2022
4,5,Gramado RS,Tudo muito sensacional equipe super atenciosa ...,jun. de 2022


## Extraindo várias páginas

In [60]:
driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')

#link da lista 
driver.get(links_hoteis[1])

soup_hotel = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

/tmp/ipykernel_73675/3056784212.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')


In [61]:
prox_page = soup_hotel.find_all("a", attrs={"class", "ui_button nav next primary"})
link_split = prox_page[0]['href'].split('-or5-')

link_hotel_paginas = []

for num_coment_inf in range(0, 21, 5):
    if num_coment_inf == 0:
        link_hotel_paginas.append("https://www.tripadvisor.com.br" + link_split[0] + '-' + link_split[1])
    else:
        link_hotel_paginas.append("https://www.tripadvisor.com.br" + link_split[0] + '-' + 'or' + str(num_coment_inf) + '-' + link_split[1])

        

df_hotel = pd.DataFrame(columns = ['nota', 'titulo', 'texto', 'data', 'ano'])
link_hotel_paginas

['https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-or5-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-or10-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-or15-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d7255333-Reviews-or20-Hotel_Valle_D_Incanto-Gramado_State_of_Rio_Grande_do_Sul.html']

In [62]:
driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')

#link da lista 
driver.get(link_hotel_paginas[1])

soup_hotel_paginas = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

/tmp/ipykernel_73675/1981762406.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')


In [63]:
_nota = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'Hlmiy F1', 'rating bubble_(..)') # NOTAS

In [64]:
_texto = raspa_pagina(soup_hotel_paginas, 'q', 'class', 'QewHA H4 _a', '<span>(.*)<\/span>') #TEXTO

In [65]:
_titulo = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'KgQgP MC _S b S6 H5 _a', '<span><span>(.*)<\/span><\/span>') #TITULO

In [66]:
_data = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'cRVSd', 'escreveu uma avaliação (.*)<\/span><\/div>') #DATA
_ano = []
for prov in _data:
    _ano.append(int(prov.split()[-1]))
_ano

[2022, 2022, 2022, 2022, 2022]

In [67]:
_ = pd.DataFrame(np.transpose(np.array([_nota, _titulo, _texto, _data, _ano])), columns = ['nota', 'titulo', 'texto', 'data', 'ano'])
df_hotel = pd.concat([df_hotel, _], ignore_index=True)

In [68]:
df_hotel

,nota,titulo,texto,data,ano
0,5,Melhor hotel romântico do Brasil,Atendimento de excelência do início ao fim. Um...,jun. de 2022,2022
1,5,Cordialidade e qualidade é o resumo!,"Desde a chegada até a saída, tudo repleto de e...",jun. de 2022,2022
2,5,Inesquecível!,"Tudo impecável, educação do pessoal do hotel f...",jun. de 2022,2022
3,5,Incrível,Simplesmente incrível… Principalmente os funci...,jun. de 2022,2022
4,5,Melhor estadia,"SUPER recomendo este hotel, que na verdade é o...",jun. de 2022,2022


## Testando para primeiro Hotel

In [69]:
driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')

#link da lista 
driver.get(links_hoteis[0])

soup_hotel = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

/tmp/ipykernel_73675/2387054048.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')


In [70]:
prox_page = soup_hotel.find_all("a", attrs={"class", "ui_button nav next primary"})
link_split = prox_page[0]['href'].split('-or5-')

link_hotel_paginas = []
limite = 7000
for num_coment_inf in range(0, limite, 5):
    if num_coment_inf == 0:
        link_hotel_paginas.append("https://www.tripadvisor.com.br" + link_split[0] + '-' + link_split[1])
    else:
        link_hotel_paginas.append("https://www.tripadvisor.com.br" + link_split[0] + '-' + 'or' + str(num_coment_inf) + '-' + link_split[1])

        

df_hotel = pd.DataFrame(columns = ['nota', 'titulo', 'texto', 'data', 'ano'])
link_hotel_paginas

['https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or5-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or10-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or15-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or20-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or25-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'https://www.tripadvisor.com.br/Hotel_Review-g303536-d13871137-Reviews-or30-Hotel_Colline_de_France-Gramado_State_of_Rio_Grande_do_Sul.html',
 'htt

In [71]:

_ano = [2022, 2022, 2022, 2022, 2022]
pagina = 0

while _ano[-1] >= 2021 and pagina < (limite - 1) / 5:
    driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')

    #link da lista 
    driver.get(link_hotel_paginas[pagina])

    soup_hotel_paginas = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    
    
    _nota = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'Hlmiy F1', 'rating bubble_(..)') # NOTAS
    _texto = raspa_pagina(soup_hotel_paginas, 'q', 'class', 'QewHA H4 _a', '<span>(.*)<\/span>') #TEXTO
    _titulo = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'KgQgP MC _S b S6 H5 _a', '<span><span>(.*)<\/span><\/span>') #TITULO
    _data = raspa_pagina(soup_hotel_paginas, 'div', 'class', 'cRVSd', 'escreveu uma avaliação (.*)<\/span><\/div>') #DATA
    _ano = []
    
    
    for prov in _data:
        if prov == 'Hoje':
            _ano.append(prov.split()[-1])
        else:
            _ano.append(int(prov.split()[-1]))
    
    
    _ = pd.DataFrame(np.transpose(np.array([_nota, _titulo, _texto, _data, _ano])), columns = ['nota', 'titulo', 'texto', 'data', 'ano'])
    df_hotel = pd.concat([df_hotel, _], ignore_index=True)
    
    print(_data[-1])
    pagina += 1

/tmp/ipykernel_73675/2448969039.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/home/murilo/.chromedriver_linux64/chromedriver')


jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
jun. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
mai. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
abr. de 2022
mar. de 2022
mar. de 2022
mar. de 2022
mar. de 2022
mar. de 2022
mar. de 2022
mar. de 2022
mar. de 2022
mar. de 2022
mar. de 2022
mar. de 2022
mar. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
fev. de 2022
jan. de 2022

In [72]:
df_hotel

,nota,titulo,texto,data,ano
0,5,Coline de France experiência única!,Me senti num castelo com personagens contempor...,jun. de 2022,2022
1,5,Experiência Diferenciada no Atendimento,Serviço extraordinário! Preocupação total com ...,jun. de 2022,2022
2,5,Sonho,"Foi um sonho realizado, eu me senti uma prince...",jun. de 2022,2022
3,5,"O HOTEL É PERFEITOOO, O ATENDIMENTO É SURREAL.",sem comentários para a perfeição que é o Colli...,jun. de 2022,2022
4,5,Sensacional,"Simplesmente o hotel é sensacional, uma experi...",jun. de 2022,2022
...,...,...,...,...,...
1115,5,Hotel colline de France,"A viagem foi impecável, equipe do hotel é muit...",jan. de 2021,2021
1116,5,Hotel imperdível,Ficamos no terceiro andar com excelente vista ...,jan. de 2021,2021
1117,5,Simplesmente fabuloso!!,"Hotel dos sonhos, não somente pelo glamour e t...",jan. de 2021,2021
1118,5,Excelente,Hotel com atendimento MARAVILHOSO! Super român...,jan. de 2021,2021
